# **Traffic Incidents optimization**

## **Context**

## **Objectives**

## **Model formulation**

## **Model implementation**

### Imports

In [69]:
import cvxpy as cp
import numpy as np
np.set_printoptions(precision=3, suppress=True)

In [76]:
i = 3 # number of agents
j = 5  # number of incidents

b = np.array([1, 2, 3, 2, 1]) # agents needed per incidents

x = cp.Variable((i, j), boolean=True) # agents-incidents matrix
z = cp.Variable(j, boolean=True) # incidents coverage vector

objective = cp.Maximize(b@z) 

constraints = [cp.sum(x, axis=0) >= cp.multiply(b, z), # required agents per incidents
               cp.sum(x, axis=1) <= 1 # one agent covers at max. one incident
               ]
               
problem = cp.Problem(objective, constraints)

problem.solve(solver="ECOS_BB")



2.0000000002438294

In [77]:
x.value

array([[1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.]])

In [78]:
z.value

array([1., 0., 0., 0., 1.])

In [75]:
b

array([1, 2, 3, 2, 1])

## **Model in practice**